<a href="https://colab.research.google.com/github/MarcosP7635/Energy/blob/main/automating_data_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download Configuration Files


In [ ]:
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import ipywidgets as widgets
!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd
def download_file_g_drive_url(url, file_path, unzip):
  file_id = url[url.index("/d/") + 3: -1*len("/view?usp=sharing")]
  gdd.download_file_from_google_drive(file_id, file_path, unzip)
def make_df(url, file_path, unzip):
  download_file_g_drive_url(url, file_path, False)
  return pd.read_csv(file_path)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df_list_url = "https://drive.google.com/file/d/1fsdQQCobfY77kX7Xh-_VYmLp41L9Shmc/view?usp=sharing"
nuclide_list_df = make_df(df_list_url, "/content/df_list.csv", False)
nuclide_list_df

,1n
0,1h
1,2h
2,3h
3,3he
4,3li
...,...
5837,293ts
5838,293og
5839,294ts
5840,294og


In [ ]:
MT_dict_url = "https://drive.google.com/file/d/1jM3O_U7z8KNcPyqUkjfK_VERDcd7N8eT/view?usp=sharing"
download_file_g_drive_url(MT_dict_url, "/content/MT_dict.csv", False)

#Install and Setup Julia

In [ ]:
def run_previous_cell_as_julia(index = -2):
     with open('script.jl', 'w') as f:
           f.write( _ih[index])
     !/content/julia-1.7.2/bin/julia script.jl
!wget https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz
!tar -xvzf julia-1.7.2-linux-x86_64.tar.gz
!touch script.jl

--2022-09-25 02:46:39--  https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz
Resolving julialang-s3.julialang.org (julialang-s3.julialang.org)... 151.101.2.49, 151.101.66.49, 151.101.130.49, ...
Connecting to julialang-s3.julialang.org (julialang-s3.julialang.org)|151.101.2.49|:443... connected.
HTTP request sent, awaiting response... 302 gce internal redirect trigger
Location: https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz [following]
--2022-09-25 02:46:39--  https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.2.112, 172.253.115.128, 172.253.122.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.2.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123295596 (118M) [application/x-tar]
Saving to: ‘julia-1.7.2-linux-x86_64.tar.gz’

julia-1.7.2-linux-

The following cell is meant to throw an error

In [ ]:
using Pkg
Pkg.add("Downloads")
Pkg.add("CSV")
Pkg.add("DataFrames")

In [ ]:
run_previous_cell_as_julia()

In [ ]:
#Import a dictionary of MTs to know what data to download
using CSV, DataFrames, Downloads
MT_df = CSV.read("/content/MT_dict.csv", DataFrame, 
                 stringtype=String)
MT_dict = Dict([])
rows, cols = size(MT_df)
for row in 1:rows
  MT_df[row, 2] = String(MT_df[row, 2][2:end-1])
  MTs = [parse(Int64, n) for n in split(MT_df[row, 2], ",")]
  for MT in MTs
      MT_dict[MT] = MT_df[row, 1] 
  end
end
MT_list = [k for k in keys(MT_dict)]
nuclide_df = CSV.read("/content/df_list.csv", DataFrame, 
                 stringtype=String)
#This function gives an exact output for each datum
function read_rational_num(input_str)
    coeff, oom = split(input_str, "E")
    return parse(Float64, coeff) * Rational(10)^parse(Float64, oom)
end
#This function gives the mass and chemical symbol for each isotope
function get_mass_name(nuclide)
    index = 1
    while (tryparse(Int64, string(nuclide[index])) != nothing && index <= length(nuclide))
        index += 1
    end
    mass = tryparse(Int64, nuclide[1:index-1])
    name =  nuclide[index:end]
    name_1 = uppercase(name[1])
    if (length(name) > 1)
        name = name_1 * name[2]
    else 
      name = name_1
    end
    return mass, name
end
#This function gives the mass and chemical symbol for each isotope
function read_tendl_file(file_path)
    file_as_vector = readlines(file_path)
    reaction = split(file_as_vector[1], " ", keepempty = false)[end]
    authors = split(file_as_vector[2], " ", keepempty = false)[3]
    energies = split(file_as_vector[4], " ", keepempty = false)[end]
    energies = parse(Int64, energies)
    x_axis_label, y_axis_label = split(file_as_vector[5], " ", keepempty = false)[2:end]
    x_vals, y_vals = Vector{Float64}(undef, energies), Vector{Float64}(undef, energies)
    for row in 1:energies
        strs = split(file_as_vector[5+row], " ", keepempty=false)
        x_vals[row], y_vals[row] = read_rational_num(strs[1]), read_rational_num(strs[2]) 
    end
    return x_vals, y_vals, x_axis_label, y_axis_label, authors, reaction
end
#This function downloads and reads the data fro a given reaction
function tendl_data_from_url(elem, mass, projectile, mt)
    zero_filled_mass = lpad(string(mass), 3, "0")
    url = "https://www-nds.iaea.org/dataexplorer/libraries/" * projectile * "/" 
    url = url * elem * zero_filled_mass * "/tendl.2019/tables/xs/" * projectile * "-"
    url = url * elem * zero_filled_mass * "-MT" * lpad(string(mt), 3, "0") * ".tendl.2019"
    return read_tendl_file(Downloads.download(url))
end
projectile = "n"
for nuclide in nuclide_df[1:end, 1]
  mass, elem = get_mass_name(nuclide)
  for mt in MT_list
    try 
      x_vals, y_vals, x_axis_label, y_axis_label, authors, reaction = tendl_data_from_url(elem, mass, projectile, mt)
      df = DataFrame()
      df[!, x_axis_label] = x_vals
      df[!, y_axis_label] = y_vals
      mt_str, mass_str = lpad(string(mt), 3, "0"), lpad(string(mass), 3, "0")
      file_path = projectile * "-" * elem * mass_str * "-" * "MT" * mt_str * ".csv"
      CSV.write(file_path, df)
      println("nuclide = ", mass, elem, " MT = ", mt, " worked :)")
    catch 
      println("nuclide = ", mass, elem, " MT = ", mt, " did not work")
    end
  end
end

In [194]:
run_previous_cell_as_julia()

nuclide = 1H MT = 152 did not work
nuclide = 1H MT = 875 did not work
nuclide = 1H MT = 50 did not work
nuclide = 1H MT = 16 did not work
nuclide = 1H MT = 891 did not work
nuclide = 1H MT = 876 did not work
nuclide = 1H MT = 38 did not work
nuclide = 1H MT = 21 did not work
nuclide = 1H MT = 17 did not work
nuclide = 1H MT = 37 did not work
nuclide = 2H MT = 152 did not work
nuclide = 2H MT = 875 did not work
nuclide = 2H MT = 50 did not work
nuclide = 2H MT = 16 worked :)
nuclide = 2H MT = 891 did not work
nuclide = 2H MT = 876 did not work
nuclide = 2H MT = 38 did not work
nuclide = 2H MT = 21 did not work
nuclide = 2H MT = 17 did not work
nuclide = 2H MT = 37 did not work
nuclide = 3H MT = 152 did not work
nuclide = 3H MT = 875 did not work
nuclide = 3H MT = 50 did not work
nuclide = 3H MT = 16 worked :)
nuclide = 3H MT = 891 did not work
nuclide = 3H MT = 876 did not work
nuclide = 3H MT = 38 did not work
nuclide = 3H MT = 21 did not work
nuclide = 3H MT = 17 did not work
nuclide 

Works :)

# :)

In [ ]:
def make_explorer_url(elem, mass, reaction):
  url = "https://www-nds.iaea.org/dataexplorer/?target_elem=" + elem 
  comma_index = reaction.index(",")
  url += "&target_mass=" + str(mass) + "&reaction=" + reaction[1:comma_index]
  url += "%2C" + reaction[comma_index+1:-1]
  return url
#explorer_url = https://www-nds.iaea.org/dataexplorer/?target_elem=H&target_mass=2&reaction=n%2Ctotal

In [ ]:
reaction = "(n,total)"
elem = "H"
mass = 2
make_explorer_url(elem, mass, reaction)

'https://www-nds.iaea.org/dataexplorer/?target_elem=H&target_mass=2&reaction=n%2Ctotal'

Example tendl url
https://www-nds.iaea.org/dataexplorer/libraries/n/H002/tendl.2019/tables/xs/n-H002-MT001.tendl.2019

In [ ]:
def make_tendl_url(elem, mass, projectile, mt):
  zero_filled_mass = str(mass).zfill(3)
  elem = elem[0].upper() + elem[1:]
  url = "https://www-nds.iaea.org/dataexplorer/libraries/" + projectile + "/" 
  url += elem + zero_filled_mass + "/tendl.2019/tables/xs/" + projectile + "-"
  url += elem + zero_filled_mass + "-MT" + str(mt).zfill(3) + ".tendl.2019"
  return url

In [ ]:
make_tendl_url("li", 7, "n", 2)

'https://www-nds.iaea.org/dataexplorer/libraries/n/Li007/tendl.2019/tables/xs/n-Li007-MT002.tendl.2019'

It works! :) 

In [ ]:
def get_tendl_data(elem, mass, projectile, mt):
  url = make_tendl_url(elem, mass, projectile, mt)
  data_as_str_arr = requests.get(url).text.split('\n')
  return data_as_str_arr
get_tendl_data("li", 7, "n", 102)

['# n + Li  7 : (n,g)     ',
 '# tendl.2019 P.G.Young                         EVAL-AUG88',
 '# uncertainties: y',
 '# # energies =   594',
 '#   E(MeV)        xs(mb)        xslow(mb)      xsupp(mb)',
 ' 1.00000E-11    2.28368E+03    2.28368E+03    2.28368E+03',
 ' 1.03223E-11    2.24774E+03    2.24774E+03    2.24774E+03',
 ' 1.06549E-11    2.21239E+03    2.21239E+03    2.21239E+03',
 ' 1.09984E-11    2.17757E+03    2.17757E+03    2.17757E+03',
 ' 1.13527E-11    2.14332E+03    2.14332E+03    2.14332E+03',
 ' 1.17186E-11    2.10959E+03    2.10959E+03    2.10959E+03',
 ' 1.20962E-11    2.07640E+03    2.07640E+03    2.07640E+03',
 ' 1.24861E-11    2.04372E+03    2.04372E+03    2.04372E+03',
 ' 1.28884E-11    2.01158E+03    2.01158E+03    2.01158E+03',
 ' 1.33038E-11    1.97992E+03    1.97992E+03    1.97992E+03',
 ' 1.37325E-11    1.94877E+03    1.94877E+03    1.94877E+03',
 ' 1.41751E-11    1.91811E+03    1.91811E+03    1.91811E+03',
 ' 1.46318E-11    1.88793E+03    1.88793E+03    1.88793E